In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
import time
import copy

In [2]:
def printTable(table):
    '''A simple way to display matrices in a more readable format'''
    for i in table:
        print(i)

In [3]:
def evaluate_util(player_row):
    if players[player_row -1][4] == True:
        restOfROI_main_role = combos[(centers - 1, wings, defenders, goalies, utils)][player_row - 1][budget_interval - players[player_row - 1][1]]
        restOfROI_util = combos[(centers, wings, defenders, goalies, utils - 1)][player_row - 1][budget_interval - players[player_row - 1][1]]

In [4]:
def get_optimal_set(players_list, budget, inc, limits):
    '''Uses a Dynamic Programming Algorithm to get the maximum number of points scored on a fantasy hockey team,
    constrained by salary cap (budget) and limits on positions (only 3 centers, 2 wings, etc.)'''
    
    ## inputs
    # players_list--a list of lists: player name, salary, expected points, position--ex:['Nathan MacKinnon', 9000, 14.94, 'C']
    # budget -- a total budget/salary cap--ex:50000
    # inc -- an increment that you go up/down by when you use the algorithm--ex:100 
    # limits -- a dictionary of constraints for each position--ex:{"C": 2,"W": 3 ,"D" : 2,"G": 1}
    
    ## first we divide all working money sums by the increment
    ## This allows us to control the granularity of the algorith
    players = [[player[0], int(int(player[1])/inc), player[2], player[3]] for player in players_list]
    budget = int(budget / inc)


    ## we get our number of players and create dictionaries for our combo table and our combo traceback table
    nPlayers = len(players) 
    combos = {}
    combos_traceback = {}
    
    ## A cassic napsack problem is only measuring 2 variables (weight and value), making it a nested list solution
    ## since this has a limitation of how many of each positions you can have, we are making a nest list solution
    ## for every possible combination of position limits
    
    ## We iterate through every possible combination of centers, wings, defenders, and goalies
    for centers in range(limits["C"] + 1):
        for wings in range(limits["W"] + 1):
            for defenders in range(limits["D"] + 1):
                for goalies in range(limits["G"] + 1):
                    
                    ## For each combination, we initialize a list of lists of placeholder values                 
                    combos[(centers, wings, defenders, goalies)] = [[-1 for i in range(budget + 1)] for i in range(nPlayers + 1)]
                    combos_traceback[(centers, wings, defenders, goalies)] = [[None for i in range(budget + 1)] for i in range(nPlayers + 1)]

                    
    ## Now that we have created players/budget tables for each combination of positions,
    ## we iterate through through each combination to start calculating our best draft option    
    for centers in range(limits["C"] + 1):
        for wings in range(limits["W"] + 1):
            for defenders in range(limits["D"] + 1):
                for goalies in range(limits["G"] + 1):
                
                
                    roster_limits = {"C":centers,"W": wings ,"D":defenders ,"G": goalies}
                    table = combos[(centers, wings, defenders, goalies)]
                    traceback = combos_traceback[(centers, wings, defenders, goalies)]
                
                    ## Filling in the base case for 0 players to choose from
                    for i in range(budget + 1):
                        table[0][i] = 0
                    
                    ## Now we incrememnt through each player
                    for player_row in range(1, nPlayers + 1):
                        
                        player_cost = players[player_row - 1][1]
                        player_points = players[player_row - 1][2]
                        player_position = players[player_row -1][3]
                        
                        
                        ## For each player, we increment through our budget
                        for budget_interval in range(budget + 1):
                        
                            ##For each acceptable player, we see if we can afford the player and if we have roster space
                            if roster_limits[player_position] > 0 and budget_interval >= player_cost:
                                
                                
                                ## To determine if we want to select a player or not, we find the higher value between:
                                ## The ROI of the player + the optimized ROI of our remaining budget and available positions
                                ## The ROI of not selecting the player
                                
                                ## Since these are tables of previously filled in dictionaries,
                                ## We just have to go to the right player limit combination, player row, and budget interval
                                ## to get the already-calculated optimized ROI
                                if player_position == "C":
                                    restOfROI = combos[(centers - 1, wings, defenders, goalies)][player_row - 1][budget_interval - player_cost]
                                elif player_position == "W":
                                    restOfROI = combos[(centers, wings -1 , defenders, goalies)][player_row - 1][budget_interval - player_cost]
                                elif player_position == "D":
                                    restOfROI = combos[(centers, wings, defenders - 1, goalies)][player_row - 1][budget_interval - player_cost]
                                elif player_position == "G":
                                    restOfROI = combos[(centers, wings, defenders, goalies - 1)][player_row - 1][budget_interval - player_cost]
                                    

                                ## We compare it for the optimal ROI if we dont sign the player
                                noSign = table[player_row - 1][budget_interval]
                
                                ## if signing the player generates more profit than not signing the player, then we add him to the table
                                if player_points + restOfROI > noSign:
                                    table[player_row][budget_interval] = player_points + restOfROI
                                    traceback[player_row][budget_interval] = True
                                
                                else:
                                    table[player_row][budget_interval] = noSign
                                    traceback[player_row][budget_interval] = False
                                    
                            ## if we cant afford the player or don't have roster room, we treat it as a noSign
                            else:
                                table[player_row][budget_interval] = table[player_row - 1][budget_interval]
                                traceback[player_row][budget_interval] = False
                                
                                
                                
    
    ## To return the optimized investment portfolio, we create an empty return list
    ## and initialize our item marker value, and a money iteration value
    ret_list = []
    players_remaining = copy.deepcopy(nPlayers)
    i = -1
    centers = copy.deepcopy(limits["C"])
    wings = copy.deepcopy(limits["W"])
    defenders = copy.deepcopy(limits["D"])
    goalies = copy.deepcopy(limits["G"])
    
    ## we loop through our table until our value (signifying the current item) is at 0
    while players_remaining > 0:
        
        ## If our traceback table tells us to buy the investment
        if combos_traceback[(centers, wings, defenders, goalies )][players_remaining][i]:
            
            ## We record the player position
            player_position = players[players_remaining - 1][3]
            
            ## We add our player's name, cost, ROI, and position to our output list
            output_row = (players[players_remaining - 1][0], players[players_remaining - 1][1] * inc, players[players_remaining - 1][2], players[players_remaining - 1][3])
            ret_list.append(output_row)
                            
            ## We increment our money iteration value backwards by the amount that we "spent" on the investment
            ## And increment our item marker value back by 1
            ## and incrememnt back the roster space
            i -= players[players_remaining - 1][1]
            players_remaining -= 1
            
            if player_position == "C":
                centers -= 1
            if player_position == "W":
                wings -= 1    
            if player_position == "D":
                defenders -= 1
            elif player_position == "G":
                goalies -= 1

        
        ## If our traceback table tells us to not buy the investment,
        ## we simply increment our item marker value back by 1
        else:
            players_remaining -= 1
    
    
    return ret_list#, combos, combos_traceback
 
    

In [5]:
players_df = pd.read_csv("DKSalaries_2_25.csv")
players_df["Position"] = players_df["Roster Position"].str[0]
players_df = players_df.drop(columns=["Name + ID","ID","Game Info","TeamAbbrev", "Roster Position"])
data_for_algo = players_df[["Name", "Salary", "AvgPointsPerGame", "Position"]].values.tolist()
data_for_algo[0]

['Nathan MacKinnon', 9000, 14.94, 'C']

In [6]:
limits = {"C": 2,"W": 3 ,"D" : 2,"G": 1}
results = get_optimal_set(data_for_algo, 50000, 190, limits)

In [7]:
results

[('Arthur Kaliyev', 2470, 11.3, 'W'),
 ('Urho Vaakanainen', 3040, 10.8, 'D'),
 ('Mats Zuccarello', 4560, 14.32, 'W'),
 ('Devon Toews', 5510, 11.85, 'D'),
 ('Calvin Petersen', 7030, 16.52, 'G'),
 ('Patrice Bergeron', 7790, 15.16, 'C'),
 ('David Pastrnak', 8740, 20.52, 'W'),
 ('Nathan MacKinnon', 8930, 14.94, 'C')]

In [8]:
def calculate_results(results):
    cap = 0
    points = 0
    for player in results:
        cap += player[1]
        points += player[2]
    
    return (cap, points)

In [9]:
calculate_results(results)

(48070, 115.41)

# Part 2 -- Drafting from the middle

In [10]:
def get_player_info(players, target):
    for player in players:
        if player[0] == target:
            return(player)

def draft_from_middle(players, already_selected, players_taken_by_other, budget, inc, limits):
    
    ## for the already selected players, we adjust our budget and 
    for selected_player in already_selected:
        player_info = get_player_info(players, selected_player)
        budget = budget - player_info[1]
        limits[player_info[3]] -= 1
    
    
    ## we remove the already selected and taken players from our player pool
    remaining_players = [player for player in players if player[0] not in players_taken_by_other]    
    remaining_players = [player for player in remaining_players if player[0] not in already_selected]    
    
    ## and use our original drafting algorithm on the updated budget, limits, and player pool
    remaining_draft = get_optimal_set(remaining_players, budget, inc, limits)
    
    ## at the very end, we add in our already selected players
    for already_selected_player in already_selected:
        remaining_draft += [get_player_info(players, already_selected_player)]
    
    return remaining_draft
    

In [11]:
players = players_df[["Name", "Salary", "AvgPointsPerGame", "Position"]].values.tolist()

already_selected = ['Nathan MacKinnon']
players_taken_by_other = ['David Pastrnak', 'Patrice Bergeron']
limits = {"C": 2,"W": 3 ,"D" : 2,"G": 1}


mid_draft = draft_from_middle(players, already_selected, players_taken_by_other, 50000, 100, limits)

In [12]:
calculate_results(mid_draft)

(49100, 106.92999999999999)

# Part 3 -- ranking optimized list of player by order of importance

Now that we can get an optimized draft that includes players already selected and players that are already taken, we are going to create an updated version of our drafting algorithm

it will:
 - run a preliminary optimized draft on all of the available players
 - for every player selected in our preliminary result, we will:
     - run anohter optimized draft as if the player was unavailable
     - we subtract it from our original optimized draft to get the players contribution
 - we rank each player by their contribution, we should first draft the player with the highest draft contribution

In [13]:
def get_player_importance(players, already_selected, players_taken_by_other, budget, inc, limits, verbose):
    
    #run a preliminary optimized draft on all of the available players
    results = draft_from_middle(players, already_selected, players_taken_by_other, budget, inc, limits)
    optimal_points = calculate_results(results)[1]

    players_with_contributions = []

    #for every player selected in our preliminary result, we will:
    for player in results:
    
        #run anohter optimized draft as if the player was unavailable
        already_selected = []
        updated_players_taken_by_other = players_taken_by_other + [player[0]]
        updated_reuslts = draft_from_middle(players, already_selected, updated_players_taken_by_other, 50000, 100, limits)
        max_points_without_player = calculate_results(updated_reuslts)[1]


        #we subtract it from our original optimized draft to get the players contribution
        player_contribution = optimal_points - max_points_without_player
        
        if verbose: 
            print("Player:",player[0]," Contribution Value:", player_contribution)
            
        players_with_contributions.append((player[0], optimal_points, max_points_without_player, player_contribution))

    #we rank each player by their contribution, we should first draft the player with the highest draft contribution
    players_ranked = sorted(players_with_contributions, key=lambda x: x[3], reverse = True)
    
    top_player = players_ranked[0]
    return(top_player, players_ranked)

In [14]:
already_selected = []
players_taken_by_other = []
limits = {"C": 2,"W": 3 ,"D" : 2,"G": 1}

start_time = time.time()
get_player_importance(players, already_selected, players_taken_by_other, 50000, 100, limits, verbose = True)
print("--- %s seconds ---" % (time.time() - start_time))

Player: Arthur Kaliyev  Contribution Value: 1.4100000000000108
Player: Urho Vaakanainen  Contribution Value: 2.6500000000000057
Player: Mats Zuccarello  Contribution Value: 3.1700000000000017
Player: Devon Toews  Contribution Value: 0.13000000000000966
Player: Calvin Petersen  Contribution Value: 0.12000000000000455
Player: Patrice Bergeron  Contribution Value: 2.8700000000000045
Player: David Pastrnak  Contribution Value: 6.140000000000001
Player: Nathan MacKinnon  Contribution Value: 1.4100000000000108
--- 52.928553342819214 seconds ---


# Part 4

Applying our Draft Algorithm to Scenareos where I have a "Flex" Player (the default in DraftKings).

In other words, I am allowed to add an additional player, regardless of the player's position

The solution is pretty simple. We run the algorithm 3 times (limits + 1 center, limits + 1 winger, limits + 1 defender). And we pick the solution with the highest amount. Since DraftKings does not do any live drafting (no worries about someone else taking our players), we don't need to draft from the middle.


In [36]:
def draft_with_util(data_for_algo, cap, inc, limits):
   
    best_combo = ''
    highest_points = -1
    util_draft = ''
    
    ## If you get one extra center
    
    limits_extra_center = copy.deepcopy(limits)
    limits_extra_center["C"] += 1
    results_extra_center = get_optimal_set(data_for_algo, cap, inc, limits_extra_center)
    total_points_extra_center = sum([player[2] for player in results_extra_center])
   
    if total_points_extra_center > highest_points:
        highest_points = total_points_extra_center
        best_combo = results_extra_center
        util_draft = "Center"
    
    print('Extra Center:', total_points_extra_center, 'expected points')

    ## if you get one extra Wing    
    limits_extra_wing = copy.deepcopy(limits)
    limits_extra_wing["W"] += 1
    results_extra_wing = get_optimal_set(data_for_algo, cap, inc, limits_extra_wing)
    total_points_extra_wing = sum([player[2] for player in results_extra_wing])

    if total_points_extra_wing > highest_points:
        highest_points = total_points_extra_wing
        best_combo = results_extra_wing
        util_draft = "Wing"

    print('Extra Wing:', total_points_extra_wing, 'expected points')

    ## if you get one extra Defender    
    limits_extra_defender = copy.deepcopy(limits)
    limits_extra_defender["D"] += 1
    
    results_extra_defender = get_optimal_set(data_for_algo, cap, inc, limits_extra_defender)
    total_points_extra_defender = sum([player[2] for player in results_extra_defender])

    if total_points_extra_defender > highest_points:
        highest_points = total_points_extra_defender
        best_combo = results_extra_defender
        util_draft = "Defender"

    print('Extra Defender:', total_points_extra_defender, 'expected points')


    ## We get the best option from our 
    best_option = max(total_points_dict, key=total_points_dict.get)
    print("The Util should be used for an aditional", util_draft)
    return best_combo, highest_points


In [37]:
limits = {"C": 2,"W": 3 ,"D" : 2,"G": 1}
draft_with_util(data_for_algo, 50000, 100, limits)

Extra Center: 120.1 expected points
Extra Wing: 122.19 expected points
Extra Defender: 121.26 expected points
The Util should be used for an aditional Wing


([('Arthur Kaliyev', 2500, 11.3, 'W'),
  ('Urho Vaakanainen', 3100, 10.8, 'D'),
  ('Filip Chytil', 3300, 8.26, 'C'),
  ('Mats Zuccarello', 4700, 14.32, 'W'),
  ('Devon Toews', 5600, 11.85, 'D'),
  ('Artemi Panarin', 6400, 13.46, 'W'),
  ('Calvin Petersen', 7200, 16.52, 'G'),
  ('Patrice Bergeron', 7900, 15.16, 'C'),
  ('David Pastrnak', 8800, 20.52, 'W')],
 122.19)